# Integrity Constraints in MySQL
Constraints in MySQL are rules applied to columns in a table to enforce data integrity and consistency. They ensure that the data entered into the database adheres to specific standards. MySQL supports several types of constraints that help maintain accuracy and reliability in your database.

## Types of Constraints in MySQL
 - NOT NULL
 - UNIQUE
 - CHECK (since MySQL 8.0.16)
 - DEFAULT
 - INDEX
 - PRIMARY KEY
 - FOREIGN KEY

### **1. NOT NULL**
The NOT NULL constraint ensures that a column cannot have a NULL value. This is used when a field should always have a value. For example, in a table of users, you wouldn't want to allow a user to have an empty email address.

``` sql
CREATE TABLE Users (
    user_id INT AUTO_INCREMENT,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL,
    PRIMARY KEY (user_id)
);
```
<hr>

### **2. UNIQUE**
The UNIQUE constraint ensures that all values in a column are different. This is useful when you want to prevent duplicate entries in a column.

``` sql
CREATE TABLE Users (
    user_id INT AUTO_INCREMENT,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    PRIMARY KEY (user_id)
);
```
<hr>

### **3. CHECK**
The CHECK constraint ensures that all values in a column satisfy a specific condition. This feature was introduced in MySQL 8.0.16.

``` sql
CREATE TABLE Products (
    product_id INT AUTO_INCREMENT,
    product_name VARCHAR(100) NOT NULL,
    price DECIMAL(10, 2) NOT NULL CHECK (price > 0),
    stock INT CHECK (stock >= 0),
    PRIMARY KEY (product_id)
);
```
<hr>

### **4. DEFAULT**
The DEFAULT constraint provides a default value for a column when no value is specified.

``` sql
CREATE TABLE Users (
    user_id INT AUTO_INCREMENT,
    username VARCHAR(50) NOT NULL,
    signup_date DATE DEFAULT CURRENT_DATE,
    PRIMARY KEY (user_id)
);
```
<hr>

### **5. INDEX**
Although not a constraint in the strictest sense, an index can be applied to improve query performance. Indexes can be added to a table to speed up the retrieval of records.

``` sql
CREATE TABLE Employees (
    emp_id INT AUTO_INCREMENT,
    emp_name VARCHAR(100),
    emp_email VARCHAR(100),
    PRIMARY KEY (emp_id),
    INDEX (emp_email)
);
```
<hr>

### **6. PRIMARY KEY**
The PRIMARY KEY constraint uniquely identifies each record in a table. A table can have only one primary key, and it can consist of one or more columns.

``` sql
CREATE TABLE Orders (
    order_id INT AUTO_INCREMENT,
    user_id INT NOT NULL,
    order_date DATE NOT NULL,
    PRIMARY KEY (order_id)
);
```

**Composite Key**

A primary key can also consist of multiple columns, referred to as a composite key.

``` sql
CREATE TABLE Enrollments (
    student_id INT NOT NULL,
    course_id INT NOT NULL,
    enrollment_date DATE NOT NULL,
    PRIMARY KEY (student_id, course_id)
);
```
<hr>

### **7. FOREIGN KEY**
The FOREIGN KEY constraint links two tables together, ensuring referential integrity by enforcing that a column (or a combination of columns) in one table matches the primary key in another.

``` sql
CREATE TABLE Orders (
    order_id INT AUTO_INCREMENT,
    user_id INT NOT NULL,
    order_date DATE NOT NULL,
    PRIMARY KEY (order_id),
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
```

**Cascading Actions with Foreign Keys**
You can also define actions when a foreign key is updated or deleted:

 - ON DELETE CASCADE: Automatically deletes records in the child table if the referenced record in the parent table is deleted.
 - ON UPDATE CASCADE: Automatically updates the child records if the referenced value in the parent table is updated.

``` sql
CREATE TABLE Orders (
    order_id INT AUTO_INCREMENT,
    user_id INT NOT NULL,
    order_date DATE NOT NULL,
    PRIMARY KEY (order_id),
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
    ON DELETE CASCADE ON UPDATE CASCADE
);
```
<hr>

# **Applying Constraints in MySQL: A Practical Example**

Let's create a database with two tables: `Categories` and `Products`, and apply constraints to these tables. Then, we'll insert some data and demonstrate how constraints enforce the rules during insertions, deletions, and updations.




#### 1. **Create the Database**
```sql
CREATE DATABASE Store;
USE Store;
```

#### 2. **Create the `Categories` Table**
```sql
CREATE TABLE Categories (
    category_id INT AUTO_INCREMENT,
    category_name VARCHAR(50) NOT NULL,
    PRIMARY KEY (category_id),
    UNIQUE (category_name)
);
```
**Explanation**:
- `category_id` is the primary key and auto-incremented.
- `category_name` is unique and cannot be `NULL`.

#### 3. **Create the `Products` Table**
```sql
CREATE TABLE Products (
    product_id INT AUTO_INCREMENT,
    product_name VARCHAR(100) NOT NULL,
    price DECIMAL(10, 2) NOT NULL CHECK (price > 0),
    stock INT CHECK (stock >= 0),
    category_id INT,
    PRIMARY KEY (product_id),
    UNIQUE (product_name),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id) 
    ON DELETE CASCADE ON UPDATE CASCADE
);
```
**Explanation**:
- `product_id` is the primary key and auto-incremented.
- `product_name` is unique and cannot be `NULL`.
- `price` has a `CHECK` constraint ensuring it is greater than 0.
- `stock` has a `CHECK` constraint ensuring it is 0 or greater.
- `category_id` is a foreign key referencing the `category_id` column in the `Categories` table, with cascading delete and update.

---



### 4. **Insert Data into `Categories` Table**
Let's insert categories into the `Categories` table.

```sql
INSERT INTO Categories (category_name)
VALUES ('Electronics'), ('Clothing'), ('Furniture');
```
---


#### 5. **Insert Data into `Products` Table**
Now, insert data into the `Products` table, linking each product to a category.

```sql
-- Correct insertion
INSERT INTO Products (product_name, price, stock, category_id)
VALUES ('Laptop', 1200.00, 50, 1), 
       ('T-Shirt', 20.00, 100, 2), 
       ('Sofa', 500.00, 10, 3);

-- Failing Insert: Price is negative, will throw an error due to CHECK constraint
INSERT INTO Products (product_name, price, stock, category_id)
VALUES ('Phone', -500.00, 20, 1);

-- Failing Insert: stock is negative, will throw an error due to CHECK constraint
INSERT INTO Products (product_name, price, stock, category_id)
VALUES ('Tablet', 800.00, -10, 1);

-- Failing Insert: NULL product_name, will throw an error due to NOT NULL constraint
INSERT INTO Products (product_name, price, stock, category_id)
VALUES (NULL, 600.00, 30, 1);

-- Failing Insert: Duplicate product_name, will throw an error due to UNIQUE constraint
INSERT INTO Products (product_name, price, stock, category_id)
VALUES ('Laptop', 1300.00, 25, 1);
```

**Explanation**:
- The first query successfully inserts valid data.
- The second query fails because the price is negative (`CHECK (price > 0)`).
- The third query fails because the stock is negative (`CHECK (stock >= 0)`).
- The fourth query fails because `product_name` is `NULL`, violating the `NOT NULL` constraint.
- The fifth query fails because `product_name` is a duplicate, violating the `UNIQUE` constraint.

---

### 6. **Demonstrate `FOREIGN KEY` with Cascading Deletes**

#### Step 1: Inserting Valid Data
```sql
INSERT INTO Products (product_name, price, stock, category_id)
VALUES ('TV', 1500.00, 30, 1);
```
This successfully inserts a `TV` into the `Products` table.

#### Step 2: Deleting a Record from the `Categories` Table
```sql
DELETE FROM Categories WHERE category_id = 1;
```
Since the `category_id` for Electronics is 1, deleting this category will also delete all associated products (`Laptop`, `Phone`, `TV`) in the `Products` table due to the `ON DELETE CASCADE` clause.

---


### 7. **Demonstrate `FOREIGN KEY` with Cascading Updates**

#### Step 1: Updating a Category's `category_id`
```sql
UPDATE Categories SET category_id = 4 WHERE category_id = 3;
```
This will update `category_id` in the `Categories` table from 3 to 4, and all products linked to this category (like the `Sofa`) will also have their `category_id` updated to 4, due to the `ON UPDATE CASCADE` clause.

---




### Summary of Constraints in Action:
1. **NOT NULL** prevents null values.
2. **UNIQUE** ensures no duplicate values.
3. **CHECK** enforces conditions like positive prices and non-negative stock.
4. **DEFAULT** provides a default value when none is specified.
5. **INDEX** improves query performance. (We will see indexes later on with detailed examples)
6. **PRIMARY KEY** uniquely identifies each row.
7. **FOREIGN KEY** ensures referential integrity and uses cascading delete and update to maintain consistency across related tables.